In [ ]:
# This is a function to generate response variable dataframe
# Decision rule: 
# If the change in "High" in 5 days is greater than 3%, we give it label "buy"
# If the change in "High" in 5 days is smaller than -3%, we give it label "sell"
# Otherwise, we give it label "hold
def create_response(start_date,end_date,stock_name):
    import pandas as pd
    import numpy as np
    import yfinance as yf
    # get stock info
    data = yf.download(stock_name, start=start_date, end=end_date,interval='1h')
    # create percentage dict
    percent_dict = {}
    for i in range(len(data)-5):
        cur_high = data.iloc[i,:]['High']
        future_high = data.iloc[i+5,:]['High']
        percent_dict[data.index[i]] = (future_high -cur_high)/cur_high
    response_df = pd.DataFrame.from_dict(percent_dict, orient='index',columns=['percent_change'])
    # Decsion rule
    bins = [-(np.inf), -0.03, 0.03, np.inf]
    names = ['sell', 'hold', 'buy']
    response_df['action'] = pd.cut(response_df['percent_change'], bins, labels=names)
    response_df = response_df.reset_index()
    return response_df

In [7]:
def download_stock_data(stock,lookforward,end_date):
    import pandas as pd
    import numpy as np
    import yfinance as yf
    data = yf.download(stock, start="2020-10-02", 
                       end=end_date,interval='60m')
    data = data.reset_index()
    data['Datetime'] = data['Datetime'].astype('str')
    time = []
    for i, row in data.iterrows():
        time.append(row['Datetime'][:-6])
    data['Time']=time
    data['Time'] = pd.to_datetime(data['Time'])
    data.drop(['Datetime','High','Low','Adj Close', 'Volume'],axis=1,inplace=True)
    cols = ['Time','Open','Close']
    data=data[cols]
    data.set_index('Time')
    time_open = dict(zip(data.Time,data.Open))
    time_close = dict(zip(data.Time,data.Close))
    Time_complete = [data.Time[0]]
    start = data.Time[0]
    end = data.Time[len(data)-1]
    from datetime import datetime, timedelta
    while start < end:
        ele = start + timedelta(hours=1)
        Time_complete.append(ele)
        start = ele
    df_time = pd.DataFrame(Time_complete,columns=['Time'])
    weekday_lst = []
    for i,row in df_time.iterrows():
        cur = row['Time']
        weekday_lst.append(cur.weekday())
    df_time['weekday'] = weekday_lst
    price = []
    from datetime import datetime, timedelta
    for i,row in df_time.iterrows():
        cur = row['Time']
        day = row['weekday']
        if cur in time_open:
            price.append(time_open[cur])
        elif ("16:30:00" in str(cur) ) :
            cur -= timedelta(hours=1)
            if cur in time_close:
                price.append(time_close[cur])
            else:
                price.append(None)
        else:
            price.append(None)
    df_time['price'] = price
    df_time = df_time.fillna(method='ffill')
    time_price = dict(zip(df_time.Time,df_time.price))
    future_price = []
    for i, row in df_time.iterrows():
        key = row['Time']+timedelta(hours=lookforward)
        day = row['weekday']
#         if key.weekday()>4:
#             if key.weekday() == 5:
#                 key += timedelta(hours=48)
#             else:
#                 key += timedelta(hours=24)
        if key in time_price:
            future_price.append(time_price[key])
        else:
            future_price.append(None)
    df_time['future_price'] = future_price
    name = 'percent_change_' + str(lookforward)
    df_time[name] = ((df_time['future_price'] / df_time['price'])-1)*100
    return df_time

In [8]:
future_24 = download_stock_data("NVDA",24,"2021-05-01")
future_48 = download_stock_data("NVDA",48,"2021-05-01")
future_72 = download_stock_data("NVDA",72,"2021-05-01")
future_96 = download_stock_data("NVDA",96,"2021-05-01")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [15]:
import pandas as pd

In [20]:
frame = [future_24.iloc[:,[0,1,2,4]],future_48.iloc[:,4],future_72.iloc[:,4],future_96.iloc[:,4]]
df = pd.concat(frame,axis=1)


In [24]:
df = df.iloc[4111,:]

In [27]:
df.info

AttributeError: 'Series' object has no attribute 'info'

In [15]:
a.tail()

,Time,weekday,price,future_price,percent_change
4106,2021-03-22 11:30:00,0,529.250000,NaN,NaN
4107,2021-03-22 12:30:00,0,533.531006,NaN,NaN
4108,2021-03-22 13:30:00,0,533.429993,NaN,NaN
4109,2021-03-22 14:30:00,0,533.799988,NaN,NaN
4110,2021-03-22 15:30:00,0,529.277527,NaN,NaN


In [2]:
nvda = download_stock_data('NVDA')
nvda.head()

[*********************100%***********************]  1 of 1 completed


,Time,weekday,price
0,2020-10-02 09:30:00,4,529.000000
1,2020-10-02 10:30:00,4,539.695007
2,2020-10-02 11:30:00,4,533.020020
3,2020-10-02 12:30:00,4,531.131287
4,2020-10-02 13:30:00,4,529.250000


In [3]:
nvda.head(60)

,Time,weekday,price
0,2020-10-02 09:30:00,4,529.000000
1,2020-10-02 10:30:00,4,539.695007
2,2020-10-02 11:30:00,4,533.020020
3,2020-10-02 12:30:00,4,531.131287
4,2020-10-02 13:30:00,4,529.250000
5,2020-10-02 14:30:00,4,528.520020
6,2020-10-02 15:30:00,4,525.500000
7,2020-10-02 16:30:00,4,522.479980
8,2020-10-02 17:30:00,4,522.479980
9,2020-10-02 18:30:00,4,522.479980


In [ ]:
time_price = dict(zip(df_time.Time,df_time.price))

In [ ]:
future_price = []
for i, row in nvda.iterrows():
    key = row['Time']+timedelta(hours=24)
    day = row['weekday']
    if key.weekday()>4:
        key += timedelta(hours=48)
    if key in time_price:
        future_price.append(time_price[key])
    else:
        future_price.append(None)

In [33]:
msft = download_stock_data('MSFT')
amd = download_stock_data('AMD')
ndaq = download_stock_data('NDAQ')
intel = download_stock_data('INTC')
qualcomm = download_stock_data('QCOM')
apple = download_stock_data('AAPL')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [38]:
msft

,Time,weekday,price
0,2020-10-02 09:30:00,4,208.000000
1,2020-10-02 10:30:00,4,210.570007
2,2020-10-02 11:30:00,4,208.419998
3,2020-10-02 12:30:00,4,207.649994
4,2020-10-02 13:30:00,4,207.559998
...,...,...,...
4106,2021-03-22 11:30:00,0,235.899994
4107,2021-03-22 12:30:00,0,236.529999
4108,2021-03-22 13:30:00,0,236.380096
4109,2021-03-22 14:30:00,0,236.310104


In [39]:
frame = [msft,amd.iloc[:,2],ndaq.iloc[:,2],intel.iloc[:,2],
         qualcomm.iloc[:,2],apple.iloc[:,2]]
stock_df = pd.concat(frame,axis=1)

In [42]:
stock_df.columns = ['Time','weekday','msft','amd','ndaq','intel',
                    'qualcomm','apple']

In [43]:
stock_df.to_csv("stock_price.csv")

,Time,weekday,msft,amd,ndaq,intel,qualcomm,apple
0,2020-10-02 09:30:00,4,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000
1,2020-10-02 10:30:00,4,210.570007,210.570007,210.570007,210.570007,210.570007,210.570007
2,2020-10-02 11:30:00,4,208.419998,208.419998,208.419998,208.419998,208.419998,208.419998
3,2020-10-02 12:30:00,4,207.649994,207.649994,207.649994,207.649994,207.649994,207.649994
4,2020-10-02 13:30:00,4,207.559998,207.559998,207.559998,207.559998,207.559998,207.559998
...,...,...,...,...,...,...,...,...
4106,2021-03-22 11:30:00,0,235.899994,235.899994,235.899994,235.899994,235.899994,235.899994
4107,2021-03-22 12:30:00,0,236.529999,236.529999,236.529999,236.529999,236.529999,236.529999
4108,2021-03-22 13:30:00,0,236.380096,236.380096,236.380096,236.380096,236.380096,236.380096
4109,2021-03-22 14:30:00,0,236.310104,236.310104,236.310104,236.310104,236.310104,236.310104


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf

In [ ]:
nvda = create_response("2020-10-02", "2021-03-19","NVDA")

In [1]:
import yfinance as yf
data = yf.download("MSFT", start="2020-10-02", end="2021-03-23",interval='60m')

[*********************100%***********************]  1 of 1 completed


In [2]:
data

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2020-10-02 09:30:00-04:00,208.000000,210.990005,207.990005,210.589996,210.589996,6766464
2020-10-02 10:30:00-04:00,210.570007,210.570007,208.350006,208.429993,208.429993,3446914
2020-10-02 11:30:00-04:00,208.419998,208.419998,205.539993,207.649994,207.649994,5986975
2020-10-02 12:30:00-04:00,207.649994,208.020004,206.470001,207.570007,207.570007,3219125
2020-10-02 13:30:00-04:00,207.559998,208.262695,206.589996,207.830002,207.830002,2662164
...,...,...,...,...,...,...
2021-03-22 11:30:00-04:00,235.899994,236.616196,235.410004,236.509995,236.509995,2820404
2021-03-22 12:30:00-04:00,236.529999,236.899994,236.229996,236.380005,236.380005,2268562
2021-03-22 13:30:00-04:00,236.380096,236.684006,236.100006,236.309998,236.309998,2023923


In [3]:
data = data.reset_index()

In [4]:
data['Datetime'] = data['Datetime'].astype('str')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 813 entries, 0 to 812
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Datetime   813 non-null    object 
 1   Open       813 non-null    float64
 2   High       813 non-null    float64
 3   Low        813 non-null    float64
 4   Close      813 non-null    float64
 5   Adj Close  813 non-null    float64
 6   Volume     813 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 44.6+ KB


In [6]:
import pandas as pd
time = []
for i, row in data.iterrows():
    time.append(row['Datetime'][:-6])

In [7]:
data['Time']=time

In [8]:
data.head(10)

,Datetime,Open,High,Low,Close,Adj Close,Volume,Time
0,2020-10-02 09:30:00-04:00,208.000000,210.990005,207.990005,210.589996,210.589996,6766464,2020-10-02 09:30:00
1,2020-10-02 10:30:00-04:00,210.570007,210.570007,208.350006,208.429993,208.429993,3446914,2020-10-02 10:30:00
2,2020-10-02 11:30:00-04:00,208.419998,208.419998,205.539993,207.649994,207.649994,5986975,2020-10-02 11:30:00
3,2020-10-02 12:30:00-04:00,207.649994,208.020004,206.470001,207.570007,207.570007,3219125,2020-10-02 12:30:00
4,2020-10-02 13:30:00-04:00,207.559998,208.262695,206.589996,207.830002,207.830002,2662164,2020-10-02 13:30:00
5,2020-10-02 14:30:00-04:00,207.830002,208.080002,206.759995,207.089996,207.089996,3037937,2020-10-02 14:30:00
6,2020-10-02 15:30:00-04:00,207.110001,207.830002,206.039993,206.179993,206.179993,3483941,2020-10-02 15:30:00
7,2020-10-05 09:30:00-04:00,207.220001,209.110001,206.979996,208.661102,208.661102,4451375,2020-10-05 09:30:00
8,2020-10-05 10:30:00-04:00,208.660095,209.250000,208.511002,209.074997,209.074997,2578595,2020-10-05 10:30:00
9,2020-10-05 11:30:00-04:00,209.080002,209.229996,208.320007,209.190002,209.190002,1910406,2020-10-05 11:30:00


In [9]:
data['Time'] = pd.to_datetime(data['Time'])

In [10]:
data.drop(['Datetime','High','Low','Adj Close', 'Volume'],axis=1,inplace=True)

In [11]:
cols = ['Time','Open','Close']
data=data[cols]

In [12]:
data.set_index('Time')

,Open,Close
Time,,
2020-10-02 09:30:00,208.000000,210.589996
2020-10-02 10:30:00,210.570007,208.429993
2020-10-02 11:30:00,208.419998,207.649994
2020-10-02 12:30:00,207.649994,207.570007
2020-10-02 13:30:00,207.559998,207.830002
...,...,...
2021-03-22 11:30:00,235.899994,236.509995
2021-03-22 12:30:00,236.529999,236.380005
2021-03-22 13:30:00,236.380096,236.309998


In [13]:
time_open = dict(zip(data.Time,data.Open))
time_close = dict(zip(data.Time,data.Close))

In [14]:
Time_complete = [data.Time[0]]
start = data.Time[0]
end = data.Time[812]
from datetime import datetime, timedelta
while start < end:
    ele = start + timedelta(hours=1)
    Time_complete.append(ele)
    start = ele

In [15]:
df_time = pd.DataFrame(Time_complete,columns=['Time'])

In [16]:
weekday_lst = []
for i,row in df_time.iterrows():
    cur = row['Time']
    weekday_lst.append(cur.weekday())

In [17]:
df_time['weekday'] = weekday_lst

In [19]:
df_time

,Time,weekday
0,2020-10-02 09:30:00,4
1,2020-10-02 10:30:00,4
2,2020-10-02 11:30:00,4
3,2020-10-02 12:30:00,4
4,2020-10-02 13:30:00,4
...,...,...
4106,2021-03-22 11:30:00,0
4107,2021-03-22 12:30:00,0
4108,2021-03-22 13:30:00,0
4109,2021-03-22 14:30:00,0


In [20]:
price = []
from datetime import datetime, timedelta
for i,row in df_time.iterrows():
    cur = row['Time']
    day = row['weekday']
    if cur in time_open:
        price.append(time_open[cur])
    elif ("16:30:00" in str(cur) ) :
        cur -= timedelta(hours=1)
        if cur in time_close:
            price.append(time_close[cur])
        else:
            price.append(None)
    else:
        price.append(None)

In [21]:
df_time['price'] = price

In [ ]:
df_time = df_time.fillna(method='ffill')

In [ ]:
df_time.head(30)

In [ ]:
time_price = dict(zip(df_time.Time,df_time.price))

In [ ]:
future_price = []
for i, row in df_time.iterrows():
    key = row['Time']+timedelta(hours=24)
    day = row['weekday']
    if key.weekday()>4:
        key += timedelta(hours=48)
    if key in time_price:
        future_price.append(time_price[key])
    else:
        future_price.append(None)

In [ ]:
df_time['future_price'] = future_price

In [ ]:
df_time.loc[0,:]

In [ ]:
df_time.loc[72,:]

In [ ]:
df_time.info()

In [ ]:
df_time.iloc[4038,:]

In [ ]:
df_time = df_time.iloc[:4039,:]

In [ ]:
df_time

In [ ]:
df_time.info()

In [ ]:
df_time['percent_change'] = ((df_time['future_price'] / df_time['price'])-1)*100

In [ ]:
df_time

In [ ]:
df_time.to_csv("./NVDA_price_1h.csv")

In [ ]:
import pandas as pd
df = pd.read_csv('NVDA_price_1h',index_col=0)
df

In [ ]:
df['Time'] = pd.to_datetime(df['Time'])

In [ ]:
df.info()

In [ ]:
df.head(15)

In [ ]:
from datetime import datetime, timedelta, date
date(2020, 10, 2) in  df['Time']

In [ ]:
df['Time'][0]

In [ ]:
date(2020, 10, 2)

In [ ]:
from datetime import datetime, timedelta
df['24_hour_later'] = df['Time'] + timedelta(hours=24)

In [ ]:
df

In [ ]:
time_price = dict(zip(df.Time,df.Open))

In [ ]:
df.iloc[0,2]

In [ ]:
df.iloc[0,2].weekday()

In [ ]:
df.iloc[0,0]

In [ ]:
df.iloc[0,0] in time_price[]

In [ ]:
future_price = []
for i, row in df.iterrows():
    key = df.iloc[i,2]
    if key.weekday()>4:
        key += timedelta(hours=48)
    if key in time_price:
        future_price.append(time_price[key])
    else:
        future_price.append(None)

In [ ]:
df.head()

In [ ]:
df['future_price'] = future_price

In [ ]:
df.head(30)

In [ ]:
df.info()